<a href="https://colab.research.google.com/github/meti-94/CodingQuestions/blob/main/Predictive%20Analysis/Transfer%20Learning/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle -q
!pip install tensorflow-addons -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d williamscott701/memotion-dataset-7k

 99% 692M/695M [00:21<00:00, 39.9MB/s]
100% 695M/695M [00:21<00:00, 33.5MB/s]


In [5]:
!unzip -qq /content/memotion-dataset-7k.zip

In [50]:
import re
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv1D, Embedding, GlobalAveragePooling1D 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [7]:
df = pd.read_csv('/content/memotion_dataset_7k/labels.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df = df.drop(columns = ['text_ocr'])
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [8]:
full_df = df.copy()
full_df.isnull().any()

image_name           False
text_corrected        True
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [9]:
clean_df = df.copy()
clean_df.dropna(inplace=True)
clean_df.isnull().any()

image_name           False
text_corrected       False
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [10]:
df = df.replace({'humour': {'not_funny': 0, 'funny': 1, 'very_funny': 2, 'hilarious':3},
            'sarcasm': {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 2, 'very_twisted': 3},
            'offensive': {'not_offensive': 0, 'slight': 1, 'very_offensive': 2, 'hateful_offensive': 3},
            'motivational': {'not_motivational': 0, 'motivational': 1},
            'overall_sentiment': {'very_negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very_positive': 4}})

In [11]:
from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [12]:
width = 100
height = 100
X_full = []
X_full_path = []
for i in tqdm(range(full_df.shape[0])):
    path = '/content/memotion_dataset_7k/images/'+full_df['image_name'][i]
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X_full.append(img)
    X_full_path.append(path)

 93%|█████████▎| 6508/6992 [00:40<00:02, 183.62it/s]/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 95%|█████████▌| 6676/6992 [00:41<00:01, 186.31it/s]/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6992/6992 [00:43<00:00, 161.78it/s]


In [13]:
width = 100
height = 100
X_cleaned = []
X_cleaned_path = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X_cleaned.append(img)
        X_cleaned_path.append(path)

100%|██████████| 6987/6987 [00:47<00:00, 148.17it/s]


In [14]:
deleted_paths = [paths for paths in X_full_path + X_cleaned_path if paths not in X_full_path or paths not in X_cleaned_path]
deleted_paths

['/content/memotion_dataset_7k/images/image_120.jpg',
 '/content/memotion_dataset_7k/images/image_4800.jpg',
 '/content/memotion_dataset_7k/images/image_6782.jpg',
 '/content/memotion_dataset_7k/images/image_6785.jpg',
 '/content/memotion_dataset_7k/images/image_6787.jpg',
 '/content/memotion_dataset_7k/images/image_6988.jpg',
 '/content/memotion_dataset_7k/images/image_6989.jpg',
 '/content/memotion_dataset_7k/images/image_6990.png',
 '/content/memotion_dataset_7k/images/image_6991.jpg',
 '/content/memotion_dataset_7k/images/image_6992.jpg']

In [15]:
delete_var = [full_df, clean_df, X_cleaned, X_full, X_cleaned_path, X_full_path]
for i in delete_var:
    del i

In [16]:
len(clean_df)

6987

In [17]:
width = 100
height = 100
X = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X.append(img)

100%|██████████| 6987/6987 [00:48<00:00, 144.33it/s]


In [18]:
X = np.array(X)

In [19]:
X.shape

(6982, 100, 100, 3)

In [20]:
rows_to_drop = ['image_120.jpg',
              'image_4800.jpg',
              'image_6782.jpg',
              'image_6785.jpg',
              'image_6787.jpg',
              'image_6988.jpg',
              'image_6989.jpg',
              'image_6990.png',
              'image_6991.jpg',
              'image_6992.jpg']

In [21]:
cleaner_df = df
cleaner_df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,3,1,0,0,4
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,0,1,0,1,4
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,2,0,0,0,3
3,image_4.png,10 Year Challenge - Sweet Dee Edition,2,2,2,1,3
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3,3,2,0,2


In [22]:
for images in rows_to_drop:
    cleaner_df.drop(cleaner_df[cleaner_df['image_name'] == images].index, inplace=True)

In [24]:
len(cleaner_df)

6982

In [25]:
cleaner_df.shape

(6982, 7)

In [26]:
np.save('image_array', X)

In [27]:
Y = cleaner_df.iloc[:,2:]
Y.shape

(6982, 5)

In [28]:
X.shape, Y.shape

((6982, 100, 100, 3), (6982, 5))

In [29]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.resnet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [30]:
base_model_1 = tf.keras.applications.ResNet50(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')
base_model_2 = tf.keras.applications.VGG16(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')

58889256/58889256 [==============================] - 2s 0us/step


In [31]:
base_model_1.trainable = False
base_model_2.trainable = False

In [32]:
global_average_layer = GlobalAveragePooling2D()

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [34]:
del X

In [35]:
image_input = tf.keras.Input(shape=(100, 100, 3), name = 'image_input')
iamge_layers = data_augmentation(image_input)
iamge_layers = preprocess_input(iamge_layers)
layer_bm_1 = base_model_1(image_input, training=False)
layer_bm_1 = Conv2D(2048, kernel_size=2,padding='valid')(layer_bm_1)
layer_bm_1 = Dense(512)(layer_bm_1)
layer_bm_2 = base_model_2(image_input, training=False)
layer_bm_2 = Dense(512)(layer_bm_2)
layers = tf.keras.layers.concatenate([layer_bm_1, layer_bm_2])
iamge_layers = global_average_layer(layer_bm_1)
image_layers = Dropout(0.2, name = 'dropout_layer')(iamge_layers)

In [36]:
cleaner_df.shape

(6982, 7)

In [37]:
def standardization(data):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: re.sub(r'\d+', '', x))
    data = data.apply(lambda x: re.sub(r'.com', '', x, flags=re.MULTILINE))
    data = data.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return data

df['text_corrected'] = standardization(cleaner_df.text_corrected)

In [38]:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(cleaner_df.text_corrected, Y, test_size = 0.2, random_state=42)

In [39]:
X_text_train

6568      how much money mark zuckerberg gave out imgflip
4004    crypto experts choose crypto investments based...
1011    me throwing away all the leonardo dicaprio mem...
4618    texan man searching for supplies and dry land ...
932     keep calm and use duct tape if you cant fix it...
                              ...                        
3773           let me tell you why thats bullshit imgflip
5193    and thats how mafia works the godfather mobile...
5228                              huh so babies cant swim
5392    leonard hofstadter a guy who actually made it ...
861     if you dont like me because im country just re...
Name: text_corrected, Length: 5585, dtype: object

In [40]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = np.asarray(X_text_train)
vectorize_layer.adapt(tf.convert_to_tensor(text_ds))

In [41]:
embedding_dim=16

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
text_layers = vectorize_layer(text_input)
text_layers = tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

text_layers = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True))(text_layers)

text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.GlobalMaxPooling1D()(text_layers)

# We add a vanilla hidden layer:
text_layers = tf.keras.layers.Dense(2048, activation="relu")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

In [42]:
concatenate = tf.keras.layers.concatenate([image_layers, text_layers], axis=1)

In [43]:
overall_layer = tf.keras.layers.Dense(2048, activation='softmax')(concatenate)

In [44]:
prediction_layer_1 = tf.keras.layers.Dense(4, activation='softmax', name = 'sarcasm')
prediction_layer_2 = tf.keras.layers.Dense(4, activation='softmax', name = 'humuor')
prediction_layer_3 = tf.keras.layers.Dense(4, activation='softmax', name = 'offensive')
prediction_layer_4 = tf.keras.layers.Dense(1, activation='sigmoid', name = 'motivational')
prediction_layer_5 = tf.keras.layers.Dense(5, activation='softmax', name = 'overall')

In [45]:
output_1 = prediction_layer_1(overall_layer)
output_2 = prediction_layer_2(overall_layer)
output_3 = prediction_layer_3(overall_layer)
output_4 = prediction_layer_4(overall_layer)
output_5 = prediction_layer_5(overall_layer)
model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])

In [46]:
import tensorflow_addons as tfa

In [51]:
base_learning_rate = 0.01
losses = {
      "humuor": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "sarcasm": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "offensive": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "motivational": tf.keras.losses.BinaryCrossentropy(from_logits=False),
      "overall": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
}
lossWeights = {
      "humuor": 1.0, 
      "sarcasm": 1.0, 
      "offensive": 1.0, 
      "motivational": 1.0,
      "overall": 1.0
}
metric = {
    "humuor": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "sarcasm": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "offensive": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "motivational": ['acc',tfa.metrics.F1Score(num_classes=1, average="micro", threshold = 0.9)],
    "overall": ['acc',tfa.metrics.F1Score(num_classes=5, average="micro", threshold = 0.9)]
}


In [52]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model-{epoch:03d}-{overall_accuracy:03f}-{val_overall_accuracy:03f}', save_format='tf', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=27, verbose=1, epsilon=1e-4, mode='min')

In [53]:
learning_rates = [0.001] # [0.1, 0.01, 0.001, 0.0001, 0.00001]
for i in learning_rates:
    model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
              loss = losses,
              loss_weights= lossWeights,
              metrics=['accuracy'])
    
    history = model.fit(x = {"image_input": X_train, "text": X_text_train},
                    y = {"sarcasm": y_train.sarcasm, 
                         "humuor": y_train.humour, 
                         "offensive": y_train.offensive, 
                         "motivational": y_train.motivational, 
                         "overall": y_train.overall_sentiment},
                    batch_size=64,
                    epochs=50,
                    verbose=2,
                    validation_split=0.2,
                    callbacks=[earlyStopping, mcp_save,reduce_lr_loss]
                   )
     

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


70/70 - 76s - loss: 6.2614 - sarcasm_loss: 1.3409 - humuor_loss: 1.3575 - offensive_loss: 1.3418 - motivational_loss: 0.6763 - overall_loss: 1.5449 - sarcasm_accuracy: 0.5002 - humuor_accuracy: 0.3534 - offensive_accuracy: 0.3899 - motivational_accuracy: 0.6437 - overall_accuracy: 0.3192 - val_loss: 6.1559 - val_sarcasm_loss: 1.3211 - val_humuor_loss: 1.3426 - val_offensive_loss: 1.3185 - val_motivational_loss: 0.6724 - val_overall_loss: 1.5013 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.3080 - lr: 0.0010 - 76s/epoch - 1s/step
Epoch 2/50


70/70 - 58s - loss: 6.0563 - sarcasm_loss: 1.2916 - humuor_loss: 1.3312 - offensive_loss: 1.2959 - motivational_loss: 0.6638 - overall_loss: 1.4738 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.3373 - val_loss: 5.9859 - val_sarcasm_loss: 1.2833 - val_humuor_loss: 1.3222 - val_offensive_loss: 1.2804 - val_motivational_loss: 0.6644 - val_overall_loss: 1.4356 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 58s/epoch - 825ms/step
Epoch 3/50


70/70 - 57s - loss: 5.9051 - sarcasm_loss: 1.2543 - humuor_loss: 1.3140 - offensive_loss: 1.2624 - motivational_loss: 0.6559 - overall_loss: 1.4184 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.8593 - val_sarcasm_loss: 1.2551 - val_humuor_loss: 1.3083 - val_offensive_loss: 1.2523 - val_motivational_loss: 0.6601 - val_overall_loss: 1.3835 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 57s/epoch - 814ms/step
Epoch 4/50


...

70/70 - 56s - loss: 5.5741 - sarcasm_loss: 1.1766 - humuor_loss: 1.2906 - offensive_loss: 1.1761 - motivational_loss: 0.6474 - overall_loss: 1.2834 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.5776 - val_sarcasm_loss: 1.2002 - val_humuor_loss: 1.2911 - val_offensive_loss: 1.1780 - val_motivational_loss: 0.6580 - val_overall_loss: 1.2502 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 56s/epoch - 799ms/step
Epoch 19/50


70/70 - 55s - loss: 5.5740 - sarcasm_loss: 1.1766 - humuor_loss: 1.2906 - offensive_loss: 1.1761 - motivational_loss: 0.6474 - overall_loss: 1.2833 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.5772 - val_sarcasm_loss: 1.2000 - val_humuor_loss: 1.2910 - val_offensive_loss: 1.1781 - val_motivational_loss: 0.6581 - val_overall_loss: 1.2500 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 55s/epoch - 792ms/step
Epoch 20/50


70/70 - 56s - loss: 5.5739 - sarcasm_loss: 1.1767 - humuor_loss: 1.2906 - offensive_loss: 1.1760 - motivational_loss: 0.6474 - overall_loss: 1.2833 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.5770 - val_sarcasm_loss: 1.1999 - val_humuor_loss: 1.2909 - val_offensive_loss: 1.1781 - val_motivational_loss: 0.6581 - val_overall_loss: 1.2500 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 56s/epoch - 794ms/step
Epoch 21/50


70/70 - 57s - loss: 5.5738 - sarcasm_loss: 1.1766 - humuor_loss: 1.2906 - offensive_loss: 1.1760 - motivational_loss: 0.6474 - overall_loss: 1.2832 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.5769 - val_sarcasm_loss: 1.1999 - val_humuor_loss: 1.2909 - val_offensive_loss: 1.1781 - val_motivational_loss: 0.6581 - val_overall_loss: 1.2499 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 57s/epoch - 815ms/step
Epoch 22/50


70/70 - 57s - loss: 5.5738 - sarcasm_loss: 1.1767 - humuor_loss: 1.2906 - offensive_loss: 1.1759 - motivational_loss: 0.6474 - overall_loss: 1.2832 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.5767 - val_sarcasm_loss: 1.1999 - val_humuor_loss: 1.2909 - val_offensive_loss: 1.1779 - val_motivational_loss: 0.6581 - val_overall_loss: 1.2499 - val_sarcasm_accuracy: 0.4736 - val_humuor_accuracy: 0.3420 - val_offensive_accuracy: 0.3688 - val_motivational_accuracy: 0.6329 - val_overall_accuracy: 0.4655 - lr: 0.0010 - 57s/epoch - 821ms/step
Epoch 23/50
70/70 - 30s - loss: 5.5737 - sarcasm_loss: 1.1767 - humuor_loss: 1.2906 - offensive_loss: 1.1760 - motivational_loss: 0.6474 - overall_loss: 1.2832 - sarcasm_accuracy: 0.5022 - humuor_accuracy: 0.3556 - offensive_accuracy: 0.3903 - motivational_accuracy: 0.6502 - overall_accuracy: 0.4373 - val_loss: 5.5771 - val_sarcasm_loss: 1.2002 - val

In [58]:
evaluate = model.evaluate(x = {"image_input": X_test, "text": X_text_test},
                    y = {"sarcasm": y_test.sarcasm, 
                         "humuor": y_test.humour, 
                         "offensive": y_test.offensive, 
                         "motivational": y_test.motivational, 
                         "overall": y_test.overall_sentiment},
                    batch_size=32,
                    verbose=2
                   )

44/44 - 3s - loss: 5.5274 - sarcasm_loss: 1.1526 - humuor_loss: 1.3076 - offensive_loss: 1.1559 - motivational_loss: 0.6475 - overall_loss: 1.2638 - sarcasm_accuracy: 0.5225 - humuor_accuracy: 0.3429 - offensive_accuracy: 0.3944 - motivational_accuracy: 0.6500 - overall_accuracy: 0.4631 - 3s/epoch - 62ms/step


In [61]:
best_model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])
best_model.load_weights('/content/model-022-0.437332-0.465533')
print('Best Model Loaded!')

Best Model Loaded!


In [63]:
best_model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
              loss = losses,
              loss_weights= lossWeights,
              metrics=['accuracy'])

In [64]:
evaluate = best_model.evaluate(x = {"image_input": X_test, "text": X_text_test},
                    y = {"sarcasm": y_test.sarcasm, 
                         "humuor": y_test.humour, 
                         "offensive": y_test.offensive, 
                         "motivational": y_test.motivational, 
                         "overall": y_test.overall_sentiment},
                    batch_size=32,
                    verbose=2
                   )

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


44/44 - 5s - loss: 5.5274 - sarcasm_loss: 1.1526 - humuor_loss: 1.3076 - offensive_loss: 1.1559 - motivational_loss: 0.6475 - overall_loss: 1.2638 - sarcasm_accuracy: 0.5225 - humuor_accuracy: 0.3429 - offensive_accuracy: 0.3944 - motivational_accuracy: 0.6500 - overall_accuracy: 0.4631 - 5s/epoch - 115ms/step


In [65]:
best_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 100)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 100, 16)      160000      ['text_vectorization[0][0]']     
                                                                                                  
 dropout (Dropout)              (None, 100, 16)      0           ['embedding[0][0]']        

In [66]:
from keras.models import Model
embedder= Model(inputs=model.input, outputs=best_model.layers[-7].output)
embedder.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 100)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 100, 16)      160000      ['text_vectorization[0][0]']     
                                                                                                  
 dropout (Dropout)              (None, 100, 16)      0           ['embedding[0][0]']        

In [67]:
embedded_X_train = embedder.predict(x = {"image_input": X_train, "text": X_text_train},
                              batch_size=32,
                              verbose=2)
embedded_X_test = embedder.predict(x = {"image_input": X_test, "text": X_text_test},
                              batch_size=32,
                              verbose=2)
print(embedded_X_train.shape, embedded_X_test.shape)

175/175 - 11s - 11s/epoch - 63ms/step
44/44 - 2s - 2s/epoch - 53ms/step
(5585, 2560) (1397, 2560)


In [68]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(embedded_X_train, y_train.iloc[:, 0].values)
y_pred = clf.predict(embedded_X_test)
print(classification_report(y_test.iloc[:, 0].values, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       335
           1       0.34      0.75      0.47       479
           2       0.33      0.26      0.29       440
           3       0.00      0.00      0.00       143

    accuracy                           0.34      1397
   macro avg       0.17      0.25      0.19      1397
weighted avg       0.22      0.34      0.25      1397



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
clf.fit(embedded_X_train, y_train.iloc[:, 0].values)
y_pred = clf.predict(embedded_X_test)
print(classification_report(y_test.iloc[:, 0].values, y_pred))

              precision    recall  f1-score   support

           0       0.25      0.16      0.19       335
           1       0.36      0.47      0.40       479
           2       0.32      0.38      0.35       440
           3       0.11      0.03      0.05       143

    accuracy                           0.32      1397
   macro avg       0.26      0.26      0.25      1397
weighted avg       0.29      0.32      0.30      1397

